In [ ]:
import pandas as pd
import numpy as np
import time
import lxml
from concurrent.futures.thread import ThreadPoolExecutor
import asyncio
import nest_asyncio
nest_asyncio.apply()

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import exceptions as e

from selenium.webdriver.chrome.options import Options  

In [ ]:
# sample crns
crns = ['1108134825', '1108134827', '1108124826', '1108134565', '1108134866']

In [ ]:
def webdriver_init(dw= '../webdriver/chromedriver'):
    chrome_options = Options()  
    chrome_options.add_argument("--headless")  
    return Chrome(dw, options=chrome_options)

In [ ]:
def base_site(browser, delay= 5, url= 'https://teht.hometax.go.kr/websquare/websquare.html?w2xPath=/ui/ab/a/a/UTEABAAA13.xml'):
    
    browser.get(url)
    browser.implicitly_wait(delay)

In [ ]:
def crn_check(browser, crn, delay= 60):
    result = [crn]
    
    browser.find_element_by_id('bsno').send_keys(crn)
    browser.implicitly_wait(0.1)
    browser.find_element_by_class_name('w2trigger').click()
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'grid2_cell_0_0')))
    except e.TimeoutException:
        pass
#         print('crn_check: {}'.format(e))
    
    table = browser.find_elements_by_xpath(("//tr[@id='row2']/td/nobr"));
    if len(table) > 0:
        for i in table:
            result.append(i.text)
    else:
        result += ['', '', '']
    return result

In [ ]:
def dataset(data):
    df = pd.DataFrame(data, columns=['input','crn','비고','조회일'])
    return df

In [ ]:
def save_csv(df):
    now = time.localtime()
    now = time.strftime("%Y%m%d%H%M%S", now)
    file_name = '../data/result/' + now + '.csv'
    df.to_csv(file_name,'\t',mode='w')

In [ ]:
def execution_all(crn):   
    browser = webdriver_init()
    base_site(browser, 0.1)
    
    result = crn_check(browser, crn)
    crns_result.append(result)
    
    browser.close()
#     print('finish: {}'.format(crn))

In [ ]:
def run_crn_checker_async(thread_workers_count, crns):
    global crns_result
    crns_result = []
    
    loop = asyncio.get_running_loop()
    executor = ThreadPoolExecutor(thread_workers_count)
    
    for crn in crns:
        loop.run_in_executor(executor, execution_all, crn)
    
    loop.run_until_complete(asyncio.gather(*asyncio.all_tasks(loop)))    